In [22]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251222 01:53:46 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [23]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())
instruments.to_csv('Token_df.csv')

expiry = instruments[(instruments['name']=='CRUDEOIL') &  (instruments['instrumenttype']=='FUTCOM') &  (instruments['exch_seg']=='MCX')]["expiry"].unique()
expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())


Next Expiry: 16JAN26


In [24]:
Symbol = 'CRUDEOIL16JAN26FUT'

def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

Token = get_option_token(Symbol)
print(f"Token: {Token}")

Token: 464925


In [25]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = Token      # NIFTY 50 Index
EXCHANGE = "MCX"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")


📈 NIFTY 50 LTP: 5105.0
📈 NIFTY 50 SPOT Price: 5100
NIFTY Change: -9.00
NIFTY % Change: -0.18%


In [26]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())
instruments.to_csv('Token_df.csv')

expiry = instruments[(instruments['name']=='CRUDEOIL') &  (instruments['instrumenttype']=='OPTFUT') &  (instruments['exch_seg']=='MCX')]["expiry"].unique()
expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())


Next Expiry: 14JAN26
